# Regularization Mode

In [1]:
from saeno.FiniteBodyForces import FiniteBodyForces
    
# initialize the object
M = FiniteBodyForces()

Set the material model:

In [2]:
from saeno.materials import SemiAffineFiberMaterial

# provide a material model
material = SemiAffineFiberMaterial(1645, 0.0008, 0.0075, 0.033)
M.setMaterialModel(material)

Load the mesh from files.

In [3]:
import numpy as np

# define the coordinates of the nodes of the mesh
# the array has to have the shape N_v x 3
R = np.array([[0., 0., 0.],  # 0
              [0., 1., 0.],  # 1
              [1., 1., 0.],  # 2
              [1., 0., 0.],  # 3
              [0., 0., 1.],  # 4
              [1., 0., 1.],  # 5
              [1., 1., 1.],  # 6
              [0., 1., 1.]]) # 7

# define the tetrahedra of the mesh
# the array has to have the shape N_t x 4
# every entry is an index referencing a verces in R (indices start with 0)
T = np.array([[0, 1, 7, 2],
              [0, 2, 5, 3],
              [0, 4, 5, 7],
              [2, 5, 6, 7],
              [0, 7, 5, 2]])

And hand the data over to the FiniteBodyForces object.

In [4]:
# provide the node data
M.setNodes(R)
# and the tetrahedron data
M.setTetrahedra(T)

Now we have to specify which displacements to fit.

In [5]:
# the displacements of the nodes which shall be fitted
# if the node is fixed (e.g. not variable) than this displacement will be fixed
# during the solving
U = np.array([[ 0.  ,  0.  ,  0.  ],  # 0
              [ 0.  ,  0.  ,  0.  ],  # 1
              [ 0.25,  0.  ,  0.  ],  # 2
              [ 0.25,  0.  ,  0.  ],  # 3
              [ 0.  ,  0.  ,  0.  ],  # 4
              [ 0.25,  0.  ,  0.  ],  # 5
              [ 0.25,  0.  ,  0.  ],  # 6
              [ 0.  ,  0.  ,  0.  ]]) # 7

# whether to fit the displacement of this node
#                     0      1      2      3      4      5      6      7
vbead = np.array([ True,  True, False, False,  True, False, False,  True])

# hand the displacements over to the class instance
M.setFoundDisplacements(U, vbead)

Now we can start the regularisation process.

In [6]:
# relax the mesh and move the "varible" nodes
M.regularize()

going to update glo f and K
updating forces and stiffness matrix finished 3.63s
|u-uf|^2 = 0.0 		perbead= 0.0
|w*f|^2  = 4.405430147324622e-28 		|u|^2 = 0.0
L = |u-uf|^2 + lambda*|w*f|^2 =  1.3216290441973865e-18
check before relax !
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.11s
Round 1  |du|= 1.7337315194255634e-17
|u-uf|^2 = 1.2023299925788246e-33 		perbead= 1.7336949706545317e-17
|w*f|^2  = 4.405430147324622e-28 		|u|^2 = 2.404659985159738e-33
L = |u-uf|^2 + lambda*|w*f|^2 =  1.3216290441973877e-18
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.07s
Round 2  |du|= 1.7337315194255585e-17
|u-uf|^2 = 4.8093199703145676e-33 		perbead= 3.467389941308801e-17
|w*f|^2  = 4.405430147324622e-28 		|u|^2 = 9.618639940638925e-33
L = |u-uf|^2 + lambda*|w*f|^2 =  1.3216290441973914e-18
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.09s
Round 3  |du|= 1.733731519425558e-17
|u-uf|^2 = 1.082096993320619e-32 		perbead= 5.20108

Round 28  |du|= 7.927174173181307e-17
|u-uf|^2 = 1.6181666811392526e-32 		perbead= 6.348365737695389e-17
|w*f|^2  = 4.3817620174792715e-28 		|u|^2 = 3.236333363408332e-32
L = |u-uf|^2 + lambda*|w*f|^2 =  1.3145286052437976e-18
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.12s
Round 29  |du|= 1.7340619160688467e-17
|u-uf|^2 = 2.466907914193746e-32 		perbead= 7.842650846728074e-17
|w*f|^2  = 1.6020272693406684e-27 		|u|^2 = 4.933815830662479e-32
L = |u-uf|^2 + lambda*|w*f|^2 =  4.8060818080220294e-18
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.10s
Round 30  |du|= 6.670926898189862e-17
|u-uf|^2 = 1.9689242496578127e-32 		perbead= 7.010787593075562e-17
|w*f|^2  = 1.639699009514837e-27 		|u|^2 = 3.9378484982486473e-32
L = |u-uf|^2 + lambda*|w*f|^2 =  4.919097028544531e-18
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.09s
Round 31  |du|= 6.776047463055426e-17
|u-uf|^2 = 2.0052053633813625e-33 		perbead= 2.2286973450

updating forces and stiffness matrix finished 0.11s
Round 57  |du|= 7.933412850135381e-17
|u-uf|^2 = 1.4393948268824958e-32 		perbead= 5.973134844930395e-17
|w*f|^2  = 4.397411884178725e-28 		|u|^2 = 2.8787896408085806e-32
L = |u-uf|^2 + lambda*|w*f|^2 =  1.3192235652536319e-18
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.12s
Round 58  |du|= 1.7347328554483427e-17
|u-uf|^2 = 2.178395836773293e-32 		perbead= 7.356202296012002e-17
|w*f|^2  = 1.598462206697741e-27 		|u|^2 = 4.356791652720894e-32
L = |u-uf|^2 + lambda*|w*f|^2 =  4.795386620093245e-18
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.09s
Round 59  |du|= 6.662942167185857e-17
|u-uf|^2 = 1.3058302452397323e-32 		perbead= 5.698894282286469e-17
|w*f|^2  = 1.6404530205585107e-27 		|u|^2 = 2.61166049020887e-32
L = |u-uf|^2 + lambda*|w*f|^2 =  4.921359061675545e-18
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.06s
Round 60  |du|= 6.778314128494188e-17
|u-uf|^2

updating forces and stiffness matrix finished 0.08s
Round 85  |du|= 6.778006446695882e-17
|u-uf|^2 = 2.009121222318703e-32 		perbead= 7.079729260324881e-17
|w*f|^2  = 2.1884135568142993e-27 		|u|^2 = 4.018242552198835e-32
L = |u-uf|^2 + lambda*|w*f|^2 =  6.565240670442918e-18
total weight:  0.018093748379241603 / 8
updating forces and stiffness matrix finished 0.06s
Round 86  |du|= 7.93069986371516e-17
|u-uf|^2 = 1.65695793381857e-32 		perbead= 6.401621481933491e-17
|w*f|^2  = 1.5922033677328138e-27 		|u|^2 = 3.313915776307991e-32
L = |u-uf|^2 + lambda*|w*f|^2 =  4.776610103198459e-18
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.06s
Round 87  |du|= 6.661485800778629e-17
|u-uf|^2 = 4.225206072082022e-33 		perbead= 3.213473817671259e-17
|w*f|^2  = 4.405719093555832e-28 		|u|^2 = 8.450412028452682e-33
L = |u-uf|^2 + lambda*|w*f|^2 =  1.3217157280667538e-18
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.06s
Round 88  |du|= 1.734071022520649

[(1.3216290441973865e-18, 0.0, 4.405430147324622e-28),
 (1.3216290441973877e-18, 1.2023299925788246e-33, 4.405430147324622e-28),
 (1.3216290441973914e-18, 4.8093199703145676e-33, 4.405430147324622e-28),
 (1.3216290441973973e-18, 1.082096993320619e-32, 4.405430147324622e-28),
 (1.3216290441974058e-18, 1.9237279881252518e-32, 4.405430147324622e-28),
 (2.156698849898989e-17, 3.005824981445249e-32, 7.188996166329953e-27),
 (1.3216290441973987e-18, 1.2182152288749619e-32, 4.405430147324622e-28),
 (6.579901073054848e-18, 1.4746678865350888e-32, 2.1933003576849445e-27),
 (1.3216290441973912e-18, 4.637921993962158e-33, 4.405430147324622e-28),
 (1.3216290441973956e-18, 9.017598868616764e-33, 4.405430147324622e-28),
 (1.3216290441974023e-18, 1.5801935728426298e-32, 4.405430147324622e-28),
 (4.810818151500614e-18, 2.4990932573389655e-32, 1.6036060505001965e-27),
 (4.9190970285445315e-18, 2.0944696061085812e-32, 1.639699009514837e-27),
 (1.3216290441973908e-18, 4.3055762889023e-33, 4.4054301473246

Now we can optinally calculate some properties of the cell, e.g. its contractility and polarisation.

In [7]:
results = M.computeForceMoments(rmax=100e-6)

d:\repositories\saeno_py\saeno\FiniteBodyForces.py:777: RuntimeWarning: invalid value encountered in true_divide
  contractility = np.sum(np.einsum("ki,ki->k", RR, f) / np.linalg.norm(RR, axis=1))
d:\repositories\saeno_py\saeno\FiniteBodyForces.py:783: RuntimeWarning: invalid value encountered in true_divide
  eR = RR / np.linalg.norm(RR, axis=1)[:, None]
d:\repositories\saeno_py\saeno\FiniteBodyForces.py:800: RuntimeWarning: invalid value encountered in true_divide
  vmid = vmid / np.linalg.norm(vmid)


In [8]:
# store the forces of the nodes
M.storeF("F.dat")
# store the positions and the displacements
M.storeRAndU("R.dat", "U.dat")
# store the center of each tetrahedron and a combined list with energies and volumina of the tetrahedrons
M.storeEandV("RR.dat", "EV.dat")

F.dat stored.
R.dat stored.
U.dat stored.
RR.dat stored.
EV.dat stored.
